In [1]:
import pandas as pd
import numpy as np
#Kebutuhan analisis regresi berganda
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
# Kebutuhan Uji Anova
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [3]:
import pandas as pd

# Simulasi pemanggilan data dari lokasi yang disimpan
file_path = r"D:\Semester 4\Metode Peramalan\Latex\Dataset Contextual\Student_Performance.csv"

# Memuat data menggunakan pandas
data = pd.read_csv(file_path)

# Menampilkan data (opsional)
print(data.head())  # Menampilkan lima baris pertama data


   Hours Studied  Previous Scores Extracurricular Activities  Sleep Hours  \
0              7               99                        Yes            9   
1              4               82                         No            4   
2              8               51                        Yes            7   
3              5               52                        Yes            5   
4              7               75                         No            8   

   Sample Question Papers Practiced  Performance Index  
0                                 1               91.0  
1                                 2               65.0  
2                                 2               45.0  
3                                 2               36.0  
4                                 5               66.0  


In [4]:
# Mengganti nilai "Yes" dan "No" dengan 1 dan 0 di kolom "Extracurricular Activities"
data['Extracurricular Activities'] = data['Extracurricular Activities'].replace({'Yes': 1, 'No': 0})

# Menampilkan data (opsional)
print(data.head())  # Menampilkan lima baris pertama data setelah perubahan

   Hours Studied  Previous Scores  Extracurricular Activities  Sleep Hours  \
0              7               99                           1            9   
1              4               82                           0            4   
2              8               51                           1            7   
3              5               52                           1            5   
4              7               75                           0            8   

   Sample Question Papers Practiced  Performance Index  
0                                 1               91.0  
1                                 2               65.0  
2                                 2               45.0  
3                                 2               36.0  
4                                 5               66.0  


In [5]:
# Membuat dummy variables untuk variabel 'Region'
region_dummies = pd.get_dummies(data['Extracurricular Activities'], prefix='Extra')

# Menyambungkan dummy variables ke dalam dataframe utama
data = pd.concat([data, region_dummies], axis=1)

# Menghapus kolom 'Region' asli karena sudah tidak diperlukan lagi setelah pembuatan dummy variables
data.drop('Extracurricular Activities', axis=1, inplace=True)

print(data.head())  # Menampilkan lima baris pertama dari dataframe yang telah dimodifikasi


   Hours Studied  Previous Scores  Sleep Hours  \
0              7               99            9   
1              4               82            4   
2              8               51            7   
3              5               52            5   
4              7               75            8   

   Sample Question Papers Practiced  Performance Index  Extra_0  Extra_1  
0                                 1               91.0    False     True  
1                                 2               65.0     True    False  
2                                 2               45.0    False     True  
3                                 2               36.0    False     True  
4                                 5               66.0     True    False  


In [6]:
print(data.dtypes)


Hours Studied                         int64
Previous Scores                       int64
Sleep Hours                           int64
Sample Question Papers Practiced      int64
Performance Index                   float64
Extra_0                                bool
Extra_1                                bool
dtype: object


In [7]:
# Mengubah tipe data boolean menjadi integer
data['Extra_0'] = data['Extra_0'].astype(int)
data['Extra_1'] = data['Extra_1'].astype(int)

# Menampilkan kembali tipe data setelah perubahan
print(data.dtypes)


Hours Studied                         int64
Previous Scores                       int64
Sleep Hours                           int64
Sample Question Papers Practiced      int64
Performance Index                   float64
Extra_0                               int32
Extra_1                               int32
dtype: object


In [8]:
# Menentukan variabel independen (X) dan variabel dependen (Y)
X = data[['Hours Studied', 'Previous Scores', 'Sleep Hours', 'Sample Question Papers Practiced', 'Extra_1']]
Y = data['Performance Index']

# Menambahkan konstanta ke variabel independen
X = sm.add_constant(X)

# Membuat model OLS
model = sm.OLS(Y, X)

# Melakukan fitting model ke data
results = model.fit()

# Menampilkan hasil summary dari model
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:      Performance Index   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                 1.757e+05
Date:                Fri, 15 Mar 2024   Prob (F-statistic):               0.00
Time:                        16:36:27   Log-Likelihood:                -21307.
No. Observations:               10000   AIC:                         4.263e+04
Df Residuals:                    9994   BIC:                         4.267e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const   

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Hitung VIF untuk setiap variabel independen
vif_data = X.assign(const=1)
vif = pd.DataFrame()
vif["Variable"] = vif_data.columns
vif["VIF"] = [variance_inflation_factor(vif_data.values, i) for i in range(vif_data.shape[1])]

print(vif)

                           Variable        VIF
0                             const  38.916475
1                     Hours Studied   1.000478
2                   Previous Scores   1.000326
3                       Sleep Hours   1.000600
4  Sample Question Papers Practiced   1.000557
5                           Extra_1   1.000802
